In [1]:
import pandas as pd
import numpy as np
import json  # Para parsear .geo
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import xgboost as xgb
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [7]:
# Definir rangos de años
years = range(2000, 2005)  # De 2000 a 2004 inclusive

# Listas para acumular dataframes procesados
modis_list = []
no_incendios_list = []

# Loop para cargar y procesar modis (incendios positivos)
for year in years:
    modis_file = f'../Incendios/modis_{year}_Peru.csv'
    if os.path.exists(modis_file):
        modis_df_year = pd.read_csv(modis_file)

        # Filtrar a región de San Martín
        san_martin_modis_year = modis_df_year[
            (modis_df_year['latitude'] >= -7.5) & (modis_df_year['latitude'] <= -5.5) &
            (modis_df_year['longitude'] >= -77.5) & (modis_df_year['longitude'] <= -75.5)
        ]

        # Agregar label
        san_martin_modis_year['incendio'] = 1

        # Seleccionar columnas relevantes
        san_martin_modis_year = san_martin_modis_year[['acq_date', 'latitude', 'longitude', 'incendio']]

        modis_list.append(san_martin_modis_year)
        print(f"Cargado y filtrado modis_{year}: {len(san_martin_modis_year)} registros")
    else:
        print(f"Archivo no encontrado: {modis_file}")

# Loop para cargar y procesar no_incendios (negativos)
for year in years:
    no_incendios_file = f'../NoIncendios/puntos_sin_incendio_sanmartin_{year}.csv'
    if os.path.exists(no_incendios_file):
        no_incendios_df_year = pd.read_csv(no_incendios_file)

        # Parsear .geo para extraer lon y lat
        no_incendios_df_year['longitude'] = no_incendios_df_year['.geo'].apply(lambda x: json.loads(x)['coordinates'][0])
        no_incendios_df_year['latitude'] = no_incendios_df_year['.geo'].apply(lambda x: json.loads(x)['coordinates'][1])

        # Filtrar a región de San Martín
        no_incendios_df_year = no_incendios_df_year[
            (no_incendios_df_year['latitude'] >= -7.5) &
            (no_incendios_df_year['latitude'] <= -5.5) &
            (no_incendios_df_year['longitude'] >= -77.5) &
            (no_incendios_df_year['longitude'] <= -75.5)
        ]

        # Agregar label
        no_incendios_df_year['incendio'] = 0

        # Seleccionar columnas relevantes
        no_incendios_df_year = no_incendios_df_year[['acq_date', 'latitude', 'longitude', 'incendio']]

        no_incendios_list.append(no_incendios_df_year)
        print(f"Cargado no_incendios_{year}: {len(no_incendios_df_year)} registros")
    else:
        print(f"Archivo no encontrado: {no_incendios_file}")

# Combinar todos los dataframes
all_modis = pd.concat(modis_list, ignore_index=True) if modis_list else pd.DataFrame()
all_no_incendios = pd.concat(no_incendios_list, ignore_index=True) if no_incendios_list else pd.DataFrame()
full_df = pd.concat([all_modis, all_no_incendios], ignore_index=True)

# Ver totales
print(f"Total incendios: {len(all_modis)}")
print(f"Total no incendios: {len(all_no_incendios)}")
print(f"Dataset completo: {len(full_df)}")
full_df.head()

Cargado y filtrado modis_2000: 57 registros
Cargado y filtrado modis_2001: 68 registros
Cargado y filtrado modis_2002: 1395 registros
Cargado y filtrado modis_2003: 2873 registros
Cargado y filtrado modis_2004: 3027 registros
Cargado no_incendios_2000: 1657 registros
Cargado no_incendios_2001: 1659 registros
Cargado no_incendios_2002: 1555 registros
Cargado no_incendios_2003: 1437 registros
Cargado no_incendios_2004: 1441 registros
Total incendios: 7420
Total no incendios: 7749
Dataset completo: 15169


C:\Users\User\AppData\Local\Temp\ipykernel_9228\3848500706.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  san_martin_modis_year['incendio'] = 1
C:\Users\User\AppData\Local\Temp\ipykernel_9228\3848500706.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  san_martin_modis_year['incendio'] = 1
C:\Users\User\AppData\Local\Temp\ipykernel_9228\3848500706.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instea

,acq_date,latitude,longitude,incendio
0,2000-11-04,-5.8207,-77.3019,1
1,2000-11-05,-5.9875,-76.8958,1
2,2000-11-05,-5.9775,-76.9008,1
3,2000-11-05,-6.4761,-77.4836,1
4,2000-11-05,-6.4902,-77.4817,1


In [8]:
# Convertir acq_date a datetime (maneja errores si hay formatos inconsistentes)
full_df['acq_date'] = pd.to_datetime(full_df['acq_date'], errors='coerce')

# Eliminar rows con fechas inválidas (si hay NaT)
full_df = full_df.dropna(subset=['acq_date'])

# Extraer features temporales
full_df['year'] = full_df['acq_date'].dt.year
full_df['month'] = full_df['acq_date'].dt.month
full_df['day'] = full_df['acq_date'].dt.day
full_df['day_of_year'] = full_df['acq_date'].dt.dayofyear

# Verificar las nuevas columnas
full_df.head()

,acq_date,latitude,longitude,incendio,year,month,day,day_of_year
0,2000-11-04,-5.8207,-77.3019,1,2000,11,4,309
1,2000-11-05,-5.9875,-76.8958,1,2000,11,5,310
2,2000-11-05,-5.9775,-76.9008,1,2000,11,5,310
3,2000-11-05,-6.4761,-77.4836,1,2000,11,5,310
4,2000-11-05,-6.4902,-77.4817,1,2000,11,5,310


In [9]:
features = ['latitude', 'longitude', 'month', 'day', 'day_of_year', 'year']
X = full_df[features]
y = full_df['incendio']

# Ver balance de clases
print(y.value_counts())

incendio
0    7749
1    7420
Name: count, dtype: int64


In [ ]:
# Calcular scale_pos_weight para desbalance
# (Aunque en este caso (7749 vs 7420) está muy balanceado)
scale_pos_weight = sum(y == 0) / sum(y == 1) if sum(y == 1) > 0 else 1
print(f"Scale pos weight: {scale_pos_weight}")

In [10]:
# Dividir en Train/Test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [12]:
# Crear modelo con scale_pos_weight
model = xgb.XGBClassifier(
    objective='binary:logistic',
    n_estimators=100,
    learning_rate=0.1,
    max_depth=5,
    random_state=42,
    # scale_pos_weight=scale_pos_weight
)

# Entrenar
model.fit(X_train, y_train)

# Predecir en test
y_pred = model.predict(X_test)

In [13]:
# Métricas
print("Accuracy (2000-2004):", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

# Matriz de confusión
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Matriz de Confusión (2000-2004)')
# Guardamos la imagen en lugar de mostrarla
plt.savefig('confusion_matrix_00-04.png')
plt.clf() # Limpiar la figura

# Feature importance
xgb.plot_importance(model)
plt.title('Importancia de Features (2000-2004)')
# Guardamos la imagen
plt.savefig('feature_importance_00-04.png')
plt.clf() # Limpiar la figura

print("Gráficos de evaluación guardados como 'confusion_matrix_00-04.png' y 'feature_importance_00-04.png'")

Accuracy (2000-2004): 0.912986156888596
              precision    recall  f1-score   support

           0       0.93      0.90      0.91      1550
           1       0.90      0.93      0.91      1484

    accuracy                           0.91      3034
   macro avg       0.91      0.91      0.91      3034
weighted avg       0.91      0.91      0.91      3034

Gráficos de evaluación guardados como 'confusion_matrix_00-04.png' y 'feature_importance_00-04.png'


<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>